In [36]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [37]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [38]:
from langchain_community.utilities import SQLDatabase

db_path = "products.db"

db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [39]:
print(db.dialect)

sqlite


In [40]:
print(db.get_usable_table_names())

['products']


In [41]:
db.run("SELECT * FROM products;")

"[(1, 'I Phone 15 pro max', 'http://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg', 'This is the latest I phone 15 pro max mobile', 2, 190000.0), (2, 'Redmi Note 9 pro', 'http://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg', 'This is redmi note 9 pro latest global version mobile', 9, 20000.0), (3, 'calculator', 'http://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg', 'casio fx caculator', 2, 1200.0), (4, 'Samsung Galaxy M30', 'https://ibb.co/sbLrxnm', 'This is samsung galaxy m30 smartphone.', 5, 20000.0), (5, 'Samsung Galaxy S22', 'https://ibb.co/Svdqcpb', 'This is samsung galaxy S22', 5, 200000.0), (6, 'Samsung Galaxy tab', 'https://ibb.co/YcfZcfm', 'This is samsung galaxy tab', 5, 110000.0), (7, 'Infinix smart 8 pro', 'https://ibb.co/s5YVpnw', 'Infinix smart 8 pro device for gaming.', 3, 30000.0), (8, 'Samsung S23 Ultra', 'https://ibb.co/DwFxTND', 'Samsung s23 ultra phone titanium.', 3,

In [42]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

In [43]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(model, db)

In [44]:
chain

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. B

In [45]:
response = chain.invoke({"question": "How many products are there"})

In [46]:
chain.invoke({
    "question": "what is the highest price of your product"
})

'Question: what is the highest price of your product\nSQLQuery: SELECT "price" FROM products ORDER BY "price" DESC LIMIT 1;'

In [47]:
chain.invoke({
    "question": "Do you have I phone 15 pro"
})

'Question: Do you have I phone 15 pro\nSQLQuery: SELECT "name" FROM products WHERE "name" LIKE \'%I Phone 15 pro%\''

In [48]:
chain.invoke({
    "question": "What is the price of redmi note 9 pro mobile phone"
})

'Here is the answer:\n\nQuestion: What is the price of redmi note 9 pro mobile phone\nSQLQuery: SELECT "price" FROM products WHERE "name" = \'Redmi Note 9 pro\''

In [49]:
print(chain.invoke({"question": "How many products are there"}))

Question: How many products are there
SQLQuery: SELECT COUNT(*) FROM products


In [50]:
import re

def extract_query(text):
    # Define the regular expression to find the SQL query part
    pattern = r'SQLQuery:\s*(.*)'
    match = re.search(pattern, text)
    
    if match:
        return match.group(1)
    else:
        return None

In [51]:
query = extract_query(response)

In [52]:
db.run(query)

'[(9,)]'

In [53]:
print(response)

Question: How many products are there
SQLQuery: SELECT COUNT(*) AS "product_count" FROM products


In [54]:
from langchain_core.runnables import RunnableLambda

query_extractor = RunnableLambda(lambda x: extract_query(x))

In [55]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query  = create_sql_query_chain(model, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | query_extractor | execute_query

In [56]:
chain

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. B

In [57]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [58]:
chain.invoke({"question": "How many products are there"})

'[(9,)]'

In [59]:
from operator import itemgetter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [60]:
answer_prompt = PromptTemplate.from_template(
    """
    You are customer support agent for an e commerce store which sell electronics products.
    You will only answer the questions based on the context below.
    Given the following user question, corresponding  SQL query, and the result of the query, generate the answer in a short and polite way as a customer support agent.
    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer:
    """
)

answer = answer_prompt | model | StrOutputParser()
answer

PromptTemplate(input_variables=['query', 'question', 'result'], template='\n    You are customer support agent for an e commerce store which sell electronics products.\n    You will only answer the questions based on the context below.\n    Given the following user question, corresponding  SQL query, and the result of the query, generate the answer in a short and polite way as a customer support agent.\n    Question: {question}\n    SQL Query: {query}\n    SQL Result: {result}\n    Answer:\n    ')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022CE464A030>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022CE463CBC0>, model_name='llama3-8b-8192', groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [61]:
query_getter = write_query | query_extractor

In [62]:
final_chain = (
    RunnablePassthrough.assign(query=query_getter).assign(result=itemgetter("query") | execute_query)
    | answer
)

In [63]:
final_chain

RunnableAssign(mapper={
  query: RunnableAssign(mapper={
           input: RunnableLambda(...),
           table_info: RunnableLambda(...)
         })
         | RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
         | PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\

In [64]:
response = final_chain.invoke({"question": "How many products are there"})

In [65]:
print(response)

Hello there! Thank you for reaching out to us. According to our system, we have a total of 9 products available in our electronic store. We're happy to help you with any questions you may have!


In [66]:
print(final_chain.invoke({"question": "Do you have Redmi note 9 pro mobile"}))

Hi there!

Thank you for reaching out to us. I'm happy to help you with your inquiry. Yes, we do have the Redmi Note 9 Pro mobile available in our store. You can check it out on our website or visit us in-store to learn more about its features and pricing. Would you like me to assist you with the product details or help you with the purchase process?


In [67]:
print(final_chain.invoke({"question": "What is the price of redmi note 9 pro mobile phone"}))

Thank you for reaching out to us! According to our records, the price of the Redmi Note 9 Pro mobile phone is $20,000. If you have any other questions or need assistance with your order, please feel free to ask!


In [68]:
print(final_chain.invoke({"question": "what is the price of redmi note 9 pro?"}))

Hi there!

Thank you for reaching out to us. I'm happy to help you with your question. The price of the Redmi Note 9 Pro is ₹20,000.00.


In [69]:
print(final_chain.invoke({"question": "Is samsung galaxy phone available in your store."}))

Thank you for reaching out to us! I'm happy to help you with your query. According to our database, yes, we do have Samsung Galaxy phones available in our store! Specifically, we have the Samsung Galaxy M30 and Samsung Galaxy S22 models in stock. If you're interested in purchasing either of these phones, please feel free to visit our website or contact us for more information.


In [70]:
print(final_chain.invoke({"question": "Do you sell bubble gum in stock?"}))

Thank you for reaching out to us! Unfortunately, we do not have any bubble gum products in stock. Our current product offerings do not include bubble gum. However, we do have a wide range of electronic products available for purchase. If you're looking for something specific, feel free to ask and I'll be happy to help!


In [71]:
print(final_chain.invoke({"question": "Where is the store located"}))

Hello! Thank you for reaching out to us. I'm happy to help you with your question. Unfortunately, it seems that our database doesn't store the physical location of our store. However, I can suggest that you visit our website and check the "About Us" or "Contact Us" section for more information on our store's location and hours of operation. If you have any further questions or concerns, please don't hesitate to ask.
